In [ ]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

import time
import re
import selenium
from selenium import webdriver as wd
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys

from tqdm import tqdm_notebook as tqdm

# 크롤링해야 할 정보

- 기사 헤드라인
- 기사 주소 url
- 업로드 날짜, 시간
- 신문사명
- 본문 내용
- 최대 4페이지 까지 (try구문 사용!)

## 2021 날짜 정보
- 31: 1,3, 5, 7, 8, 10, 12
- 30: 4, 6, 9, 11
- 28: 2

In [ ]:
day=[str(i).zfill(2) for i in range(1,32)] #### User-Change ####

In [ ]:
day

['01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31']

In [ ]:
#### User-Change ####
url_info="https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=202112" #### User-Change ####
url_lst=[]
for d in day:
    url=url_info+str(d)
    for num in range(1,5):
        url_lst.append(url+'&page='+str(num))

In [ ]:
url_lst
# https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20210101&page=2

['https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211201&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211201&page=2',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211201&page=3',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211201&page=4',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211202&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211202&page=2',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211202&page=3',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211202&page=4',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211203&page=1',
 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211203&page=2',


In [ ]:
len(url_lst)

124

In [ ]:
### soup으로 다음과 같은 정보부터 가져오기
# 기사 헤드라인
# 기사 주소 url
# 업로드 날짜, 시간
# 신문사명



title=[]
ar_url=[]
day=[]
time_=[]
com=[]


for u in url_lst:
    print(u)
    page=requests.get(u,headers={'User-Agent':'Mozilla/5.0'})
    soup = BeautifulSoup(page.content, "html.parser")

    #### 상위 10개 기사 ####
    top10=soup.select('#main_content > div.list_body.newsflash_body > ul.type06_headline > li')
    for i in range(0,len(top10)):
        try:
            # title
            title.append(top10[i].find_all('a')[1].text.replace('\n','').replace('\t',''))
            # ar_url
            ar_url.append(top10[i].find_all('a')[1]['href'])
            # day
            day.append(top10[i].find_all('span','date')[0].text.split('. ')[0])
            #time
            time_.append(top10[i].find_all('span','date')[0].text.split('. ')[1])
            #com
            com.append(top10[i].find_all('span','writing')[0].text)
        except:
            print('Out of Range: ',i)
            # title
            title.append(top10[i].find_all('a')[0].text.replace('\n','').replace('\t',''))
            # ar_url
            ar_url.append(top10[i].find_all('a')[0]['href'])
            # day
            day.append(top10[i].find_all('span','date')[0].text.split('. ')[0])
            # time
            time_.append(top10[i].find_all('span','date')[0].text.split('. ')[1])
            # com
            com.append(top10[i].find_all('span','writing')[0].text)  

    #### 하위 10개 기사 ####
    top20=soup.select('#main_content > div.list_body.newsflash_body > ul.type06 > li')
    for i in range(0,len(top20)):
        try:
            # title
            title.append(top20[i].find_all('a')[1].text.replace('\n','').replace('\t',''))
            # ar_url
            ar_url.append(top20[i].find_all('a')[1]['href'])
            # day
            day.append(top20[i].find_all('span','date')[0].text.split('. ')[0])
            #time
            time_.append(top20[i].find_all('span','date')[0].text.split('. ')[1])
            #com
            com.append(top20[i].find_all('span','writing')[0].text)

        except:
            print('Out of Range: ',i)
            # title
            title.append(top20[i].find_all('a')[0].text.replace('\n','').replace('\t',''))
            # ar_url
            ar_url.append(top20[i].find_all('a')[0]['href'])
            # day
            day.append(top20[i].find_all('span','date')[0].text.split('. ')[0])
            # time
            time_.append(top20[i].find_all('span','date')[0].text.split('. ')[1])
            #com
            com.append(top20[i].find_all('span','writing')[0].text)
            

            

https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211201&page=1
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211201&page=2
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211201&page=3
Out of Range:  2
Out of Range:  0
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211201&page=4
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211202&page=1
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211202&page=2
Out of Range:  7
Out of Range:  9
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211202&page=3
Out of Range:  0
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211202&page=4
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211203&page=1
https://news.naver.com/main/list.naver?mode=LS2D&mi

https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211220&page=3
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211220&page=4
Out of Range:  6
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211221&page=1
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211221&page=2
Out of Range:  0
Out of Range:  2
Out of Range:  4
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211221&page=3
Out of Range:  4
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211221&page=4
Out of Range:  8
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211222&page=1
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211222&page=2
https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=310&sid1=101&date=20211222&page=3
https://news.naver.com/main/list.n

In [ ]:
print(len(title))
print(len(ar_url))
print(len(day))
print(len(time_))
print(len(com))

2419
2419
2419
2419
2419


In [ ]:
month1=pd.DataFrame({'title':title,'url':ar_url,'day':day,'time':time_,'company':com})
month1

,title,url,day,time,company
0,해외 다녀온 한국인 5명 오미크론 감염…지역사회에 퍼졌다(종합2보),https://n.news.naver.com/mnews/article/421/000...,2021.12.01,오후 11:50,뉴스1
1,[포토] 영등포구 자원봉사 유공자 표창,https://n.news.naver.com/mnews/article/018/000...,2021.12.01,오후 11:04,이데일리
2,국내도 오미크론 뚫렸다…나이지리아 방문 부부 등 5명 감염(종합),https://n.news.naver.com/mnews/article/421/000...,2021.12.01,오후 10:44,뉴스1
3,"""샤넬이 만들었대""…이 추운 날 3시간 만에 수백명 몰렸다",https://n.news.naver.com/mnews/article/015/000...,2021.12.01,오후 10:14,한국경제
4,"""겨울아 어서와 신나게 놀아줄게""…올 겨울 든든한 키즈 패션",https://n.news.naver.com/mnews/article/009/000...,2021.12.01,오후 10:05,매일경제
...,...,...,...,...,...
2414,"한촌설렁탕, '이연에프엔씨' 얼큰설렁탕 간편식 1000팩 기부",https://n.news.naver.com/mnews/article/417/000...,2021.12.31,오후 2:03,머니S
2415,"유재석도 ""혹시 당근?"" 외치더니 '대박'…거래 폭발했다",https://n.news.naver.com/mnews/article/015/000...,2021.12.31,오후 1:54,한국경제
2416,"맛x영양…동경, 국산 단호박 떡·죽 선봬",https://n.news.naver.com/mnews/article/003/001...,2021.12.31,오후 1:45,뉴시스
2417,소비자물가지수 2.5% 상승…10년만에 최고치,https://n.news.naver.com/mnews/article/001/001...,2021.12.31,오후 1:44,연합뉴스


In [ ]:
month1.isna().sum()

title      0
url        0
day        0
time       0
company    0
dtype: int64

# contents

In [ ]:
contents=[]

for u in tqdm(list(month1['url']),desc='tqdm'):
    #print(u)
    page=requests.get(u,headers={'User-Agent':'Mozilla/5.0'})
    soup = BeautifulSoup(page.content, "html.parser")
    try:
        contents.append(soup.find('div',{'class':'newsct_article _article_body'}).text)
    except:
        print(u)
        contents.append(np.nan)

https://n.news.naver.com/mnews/article/001/0012820865?sid=101
https://n.news.naver.com/mnews/article/001/0012828410?sid=101
https://n.news.naver.com/mnews/article/001/0012810730?sid=101



In [ ]:
print(len(title))
print(len(ar_url))
print(len(day))
print(len(time_))
print(len(com))
print(len(contents))

2419
2419
2419
2419
2419
2419


In [ ]:
month1['contents']=contents

In [ ]:
month1

,title,url,day,time,company,contents
0,해외 다녀온 한국인 5명 오미크론 감염…지역사회에 퍼졌다(종합2보),https://n.news.naver.com/mnews/article/421/000...,2021.12.01,오후 11:50,뉴스1,\n\n나이지리아 방문 부부 2명 외 접촉자 3명도 오미크론 변이 검출3일부터 백신...
1,[포토] 영등포구 자원봉사 유공자 표창,https://n.news.naver.com/mnews/article/018/000...,2021.12.01,오후 11:04,이데일리,\n\n\n\n\n\n[이데일리 김태형 기자] 채현일(오른쪽 다섯번째) 영등포구청장...
2,국내도 오미크론 뚫렸다…나이지리아 방문 부부 등 5명 감염(종합),https://n.news.naver.com/mnews/article/421/000...,2021.12.01,오후 10:44,뉴스1,\n\n부부 자녀·지인 가족 등 추가 분석 중…해외입국 확진자 중 2명 추가오미크론...
3,"""샤넬이 만들었대""…이 추운 날 3시간 만에 수백명 몰렸다",https://n.news.naver.com/mnews/article/015/000...,2021.12.01,오후 10:14,한국경제,"\n\n유통가 겨울 스포츠 '스케이트 마케팅' 롯데백화점, 샤넬과 아이스링크 개장…..."
4,"""겨울아 어서와 신나게 놀아줄게""…올 겨울 든든한 키즈 패션",https://n.news.naver.com/mnews/article/009/000...,2021.12.01,오후 10:05,매일경제,\n\n올겨울 든든한 키즈 패션화사한 컬러에 스타일리시한다운재킷으로 보온·방한 효과...
...,...,...,...,...,...,...
2414,"한촌설렁탕, '이연에프엔씨' 얼큰설렁탕 간편식 1000팩 기부",https://n.news.naver.com/mnews/article/417/000...,2021.12.31,오후 2:03,머니S,"\n\n\t\t\t한촌설렁탕, 육수당 등 프랜차이즈와 간편식 사업을 진행하고 있는 ..."
2415,"유재석도 ""혹시 당근?"" 외치더니 '대박'…거래 폭발했다",https://n.news.naver.com/mnews/article/015/000...,2021.12.31,오후 1:54,한국경제,\n\n당근마켓 중고 거래 게시글 전년比 30% 증가5240만 그루 소나무 심은 효...
2416,"맛x영양…동경, 국산 단호박 떡·죽 선봬",https://n.news.naver.com/mnews/article/003/001...,2021.12.31,오후 1:45,뉴시스,"\n\n기사내용 요약찹쌀떡·크림떡·호박죽 등 3종단호박, 2시간 이상 끓여 풍미↑찹..."
2417,소비자물가지수 2.5% 상승…10년만에 최고치,https://n.news.naver.com/mnews/article/001/001...,2021.12.31,오후 1:44,연합뉴스,\n\n\n\n\n\n (서울=연합뉴스) 강민지 기자 = 통계청이 2021년 ...


In [ ]:
month1.isna().sum()

title       0
url         0
day         0
time        0
company     0
contents    3
dtype: int64

In [ ]:
month1.to_csv('month12.csv',encoding='utf-8',index=False)

In [ ]:
month11=pd.read_csv('month11.csv')
month11.head(10)

,title,url,day,time,company
0,'곰표맥주'도 간다…콜라보로 뜬 수제맥주들 IPO 준비 착착,https://n.news.naver.com/mnews/article/015/000...,2021.11.01,오후 10:02,한국경제
1,"""수지·아이유도 입는대""…벌써부터 대박 조짐",https://n.news.naver.com/mnews/article/015/000...,2021.11.01,오후 9:02,한국경제
2,[데일리안 오늘뉴스 종합] '위드 코로나 첫날' 전국 오후 6시까지 1159명 신...,https://n.news.naver.com/mnews/article/119/000...,2021.11.01,오후 8:57,데일리안
3,"2021년 10월 프랜차이즈 정보공개서 신규등록 1,229개 명단 공개 (2)",https://n.news.naver.com/mnews/article/417/000...,2021.11.01,오후 8:39,머니S
4,"요기요, 멤버십 할인 구독 서비스 '요기패스' 출시",https://n.news.naver.com/mnews/article/421/000...,2021.11.01,오후 8:12,뉴스1
5,"2021년 10월 프랜차이즈 정보공개서 신규등록 1,229개 명단 공개 (1)",https://n.news.naver.com/mnews/article/417/000...,2021.11.01,오후 8:03,머니S
6,"[포토] 렉스톤보청기, 신제출 9종 출시",https://n.news.naver.com/mnews/article/018/000...,2021.11.01,오후 7:52,이데일리
7,[내일 날씨] 아침 기온 5도 이하로 '뚝'…큰 일교차,https://n.news.naver.com/mnews/article/119/000...,2021.11.01,오후 7:30,데일리안
8,부활하고 있는 김선호 광고…도미노 피자는 '이것' 택했다,https://n.news.naver.com/mnews/article/018/000...,2021.11.01,오후 7:04,이데일리
9,"교촌치킨, '사랑아 교촌해' 유튜브 100만뷰 돌파 … ﻿‘맥스서밋 2021’ 어...",https://n.news.naver.com/mnews/article/417/000...,2021.11.01,오후 7:03,머니S


In [ ]:
month11.tail(10)

,title,url,day,time,company
2349,내년부터 상속세 최대 10년까지 나눠낸다,https://n.news.naver.com/mnews/article/001/001...,2021.11.30,오후 4:36,연합뉴스
2350,"아영FBC, 파이퍼하이직 오흐 세리 1971 한정 판매",https://n.news.naver.com/mnews/article/468/000...,2021.11.30,오후 4:32,스포츠서울
2351,강희석 SSG닷컴 대표 “내년 온·오프라인 시너지 완성 원년될 것”,https://n.news.naver.com/mnews/article/018/000...,2021.11.30,오후 4:31,이데일리
2352,"코니바이에린, 베이비 라인 신제품 3종 출시",https://n.news.naver.com/mnews/article/119/000...,2021.11.30,오후 4:26,데일리안
2353,"""툭 터놓고 톡 합시다"" 쓱닷컴, '오픈 톡' 열고 비대면 소통 강화",https://n.news.naver.com/mnews/article/421/000...,2021.11.30,오후 4:24,뉴스1
2354,"배민에 '기본 반찬 안 받기' 기능 생긴다…""환경 보호""",https://n.news.naver.com/mnews/article/003/001...,2021.11.30,오후 4:23,뉴시스
2355,엔지켐생명과학 “구강점막염 신약 기술이전 자신”...글로벌 3상 추진,https://n.news.naver.com/mnews/article/018/000...,2021.11.30,오후 4:19,이데일리
2356,"현지에서 먹힐까?…하이트진로, ‘햄소(햄버거+소주)’로 英시장 공략",https://n.news.naver.com/mnews/article/020/000...,2021.11.30,오후 4:13,동아일보
2357,"머지포인트, 모스버거 등 오프라인 결제 중단… 사업 축소 수순",https://n.news.naver.com/mnews/article/366/000...,2021.11.30,오후 4:12,조선비즈
2358,"“AI 코치가 스윙 교정”…골프존, 서비스 차별화 속도",https://n.news.naver.com/mnews/article/366/000...,2021.11.30,오후 4:08,조선비즈


'https://n.news.naver.com/mnews/article/018/0004843067?sid=101'

# merged

In [ ]:
month=[i for i in range(1,13)]
month

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

In [ ]:
mon1=pd.read_csv('month1.csv')
mon1.isna().sum()

title       0
url         0
day         0
time        0
company     0
contents    1
dtype: int64

In [ ]:
df=pd.read_csv('month1.csv')

for i in month[1:]:
    m=pd.read_csv('month'+str(i)+'.csv')
    print(i)
    print(m.isna().sum())
    print('------------')
    df=pd.concat([df,m])

2
title       0
url         0
day         0
time        0
company     0
contents    4
dtype: int64
------------
3
title      0
url        0
day        0
time       0
company    0
dtype: int64
------------
4
title      0
url        0
day        0
time       0
company    0
dtype: int64
------------
5
title      0
url        0
day        0
time       0
company    0
dtype: int64
------------
6
title      0
url        0
day        0
time       0
company    0
dtype: int64
------------
7
title      0
url        0
day        0
time       0
company    0
dtype: int64
------------
8

C:\Users\koh99\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  



title      0
url        0
day        0
time       0
company    0
dtype: int64
------------
9
title      0
url        0
day        0
time       0
company    0
dtype: int64
------------
10
title      0
url        0
day        0
time       0
company    0
dtype: int64
------------
11
title      0
url        0
day        0
time       0
company    0
dtype: int64
------------
12
title      0
url        0
day        0
time       0
company    0
dtype: int64
------------


In [ ]:
df

,company,contents,day,time,title,url
0,동아일보,\n\n\n\n\n\n2021년 신축년(辛丑年) 새해 첫 날인 1일 오후 서울 중구...,2021.01.01,오후 9:39,‘확진자 폭발적 증가는 막았다’…업종별 숨통 열어줄 듯,https://n.news.naver.com/mnews/article/020/000...
1,연합뉴스,\n\n\n\n\n\n유시민 노무현재단 이사장사람사는세상노무현재단 유튜브 캡처 [재...,2021.01.01,오후 9:14,"유시민 ""부동산세 헐렁하단 지적…불로소득 더 과세해야""",https://n.news.naver.com/mnews/article/001/001...
2,데일리안,\n\n\n\n\n\nⓒ한만두식품한만두식품이 다가오는 설 명절을 맞아 2021 설 ...,2021.01.01,오후 9:04,"한만두식품, 설 선물세트 출시",https://n.news.naver.com/mnews/article/119/000...
3,뉴스1,"\n\n'수도권 2.5단계/비수도권 2단계', '연말연시 특별방역' 3일 시한 추가...",2021.01.01,오후 7:09,"2일 거리두기 조정 발표…3단계는 배제, 현 단계 연장할 듯",https://n.news.naver.com/mnews/article/421/000...
4,뉴스1,"\n\n외국인 공항 입국 8일부터, 항만 승선 15일부터 음성 확인서 의무화PCR음...",2021.01.01,오후 4:07,'변이 코로나' 전 세계 비상…모든 입국 외국인에 'PCR음성' 확인,https://n.news.naver.com/mnews/article/421/000...
...,...,...,...,...,...,...
2414,머니S,NaN,2021.12.31,오후 2:03,"한촌설렁탕, '이연에프엔씨' 얼큰설렁탕 간편식 1000팩 기부",https://n.news.naver.com/mnews/article/417/000...
2415,한국경제,NaN,2021.12.31,오후 1:54,"유재석도 ""혹시 당근?"" 외치더니 '대박'…거래 폭발했다",https://n.news.naver.com/mnews/article/015/000...
2416,뉴시스,NaN,2021.12.31,오후 1:45,"맛x영양…동경, 국산 단호박 떡·죽 선봬",https://n.news.naver.com/mnews/article/003/001...
2417,연합뉴스,NaN,2021.12.31,오후 1:44,소비자물가지수 2.5% 상승…10년만에 최고치,https://n.news.naver.com/mnews/article/001/001...


In [ ]:
df.isna().sum()

company         0
contents    23977
day             0
time            0
title           0
url             0
dtype: int64

In [ ]:
mon1=pd.read_csv('month1.csv')
mon2=pd.read_csv('month2.csv')
mon3=pd.read_csv('month3.csv')
print(mon1.shape)
print(mon2.shape)
print(mon3.shape)
pd.concat([mon1,mon2])

(2348, 6)
(2159, 6)
(2404, 5)


,title,url,day,time,company,contents
0,‘확진자 폭발적 증가는 막았다’…업종별 숨통 열어줄 듯,https://n.news.naver.com/mnews/article/020/000...,2021.01.01,오후 9:39,동아일보,\n\n\n\n\n\n2021년 신축년(辛丑年) 새해 첫 날인 1일 오후 서울 중구...
1,"유시민 ""부동산세 헐렁하단 지적…불로소득 더 과세해야""",https://n.news.naver.com/mnews/article/001/001...,2021.01.01,오후 9:14,연합뉴스,\n\n\n\n\n\n유시민 노무현재단 이사장사람사는세상노무현재단 유튜브 캡처 [재...
2,"한만두식품, 설 선물세트 출시",https://n.news.naver.com/mnews/article/119/000...,2021.01.01,오후 9:04,데일리안,\n\n\n\n\n\nⓒ한만두식품한만두식품이 다가오는 설 명절을 맞아 2021 설 ...
3,"2일 거리두기 조정 발표…3단계는 배제, 현 단계 연장할 듯",https://n.news.naver.com/mnews/article/421/000...,2021.01.01,오후 7:09,뉴스1,"\n\n'수도권 2.5단계/비수도권 2단계', '연말연시 특별방역' 3일 시한 추가..."
4,'변이 코로나' 전 세계 비상…모든 입국 외국인에 'PCR음성' 확인,https://n.news.naver.com/mnews/article/421/000...,2021.01.01,오후 4:07,뉴스1,"\n\n외국인 공항 입국 8일부터, 항만 승선 15일부터 음성 확인서 의무화PCR음..."
...,...,...,...,...,...,...
2154,"[알아봅시다]패딩 표백제 피하고, 플리스 미지근한 물에",https://n.news.naver.com/mnews/article/003/001...,2021.02.28,오전 6:00,뉴시스,\n\n겨울철 의류 관리하는 법\n\n\n\n[서울=뉴시스] (사진=애경산업 제공)...
2155,"정치권서 ""세금 걷어 복지 늘리자"" 봇물…누가 부담?",https://n.news.naver.com/mnews/article/001/001...,2021.02.28,오전 5:51,연합뉴스,"\n\n학계, 증세엔 공감…부자증세·보편증세 의견 갈려\n\n\n\n[그래픽] 국가..."
2156,'봄 맞이 가전 사볼까'…하이마트 '빅하트 세일',https://n.news.naver.com/mnews/article/003/001...,2021.02.28,오전 5:00,뉴시스,\n\n\n\n\n\n[서울=뉴시스] 손정빈 기자 = 롯데하이마트가 온라인쇼핑몰에서...
2157,"[장수브랜드 탄생비화]빙그레 바나나맛우유, 매출 2천억 비결은 '한국적'",https://n.news.naver.com/mnews/article/003/001...,2021.02.28,오전 5:00,뉴시스,\n\n1974년 출시후 국내 1위 가공우유한국적 정서 담은 용기가 브랜드 상징아이...


In [ ]:
2348+2159

4507